<a href="https://colab.research.google.com/github/DarleneJD/ACOPF/blob/main/13barrasOPF_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [2]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [3]:
# if not (shutil.which("glpk") or os.path.isfile("glpk")):
#     if "google.colab" in sys.modules:
#         !apt-get install -y -qq glpk-utils
#     else:
#         try:
#             !conda install -c conda-forge glpk
#         except:
#             pass

In [4]:
!git clone https://github.com/DarleneJD/ACOPF.git

Cloning into 'ACOPF'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 25 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 1001.28 KiB | 6.76 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
import math
from pathlib import Path
import pandas as pd
import pyomo.environ as pyo
from collections import defaultdict


In [ ]:
# -----------------------------
# Voltage limits (como você vinha usando no Pyomo)
# -----------------------------
bus_voltage_min = {0: 0.85, 1: 0.85, 2: 0.92, 3: 0.92}
bus_voltage_max = {0: 1.15, 1: 1.15, 2: 1.08, 3: 1.08}

# AMPL params (do seu AMPL)
p_gen_upper = 6.0


In [ ]:
# -----------------------------
# Parsing helpers (SEUS)
# -----------------------------
def read_bus_file(path: str | Path) -> pd.DataFrame:
    """
    Expected 17 columns (from AMPL printf)
    """
    df = pd.read_csv(path, sep=r"\s+", engine="python", quotechar='"', header=None)
    if df.shape[1] != 17:
        raise ValueError(f"BUS file expected 17 columns, got {df.shape[1]}")
    df.columns = [
        "bus", "bus_type", "bus_name", "bus_voltage0", "bus_angle0_deg",
        "bus_p_gen", "bus_q_gen", "bus_q_min", "bus_q_max",
        "bus_p_load", "bus_q_load", "bus_g_shunt", "bus_b_shunt0",
        "bus_b_shunt_min", "bus_b_shunt_max", "bus_b_dispatch", "bus_area",
    ]
    df["bus"] = df["bus"].astype(int)
    df["bus_type"] = df["bus_type"].astype(int)
    df["bus_b_dispatch"] = df["bus_b_dispatch"].astype(int)
    df["bus_area"] = df["bus_area"].astype(int)
    return df


def read_branch_file(path: str | Path) -> pd.DataFrame:
    """
    Expected 13 columns (from AMPL printf)
    """
    df = pd.read_csv(path, sep=r"\s+", engine="python", header=None)
    if df.shape[1] != 13:
        raise ValueError(f"BRANCH file expected 13 columns, got {df.shape[1]}")
    df.columns = [
        "l", "k", "m", "branch_type", "branch_r", "branch_x", "branch_c",
        "branch_tap0", "branch_tap_min0", "branch_tap_max0",
        "branch_def0_deg", "branch_def_min_deg", "branch_def_max_deg",
    ]
    df[["l", "k", "m", "branch_type"]] = df[["l", "k", "m", "branch_type"]].astype(int)
    return df

# -----------------------------
# Caminhos dos arquivos
# AJUSTE se necessário
# -----------------------------
bus_path    = Path("ACOPF/140new_caso4.bus")
branch_path = Path("ACOPF/140r.branch")

# -----------------------------
# Leitura
# -----------------------------
bus_df = read_bus_file(bus_path)
branch_df = read_branch_file(branch_path)

BUS = bus_df.bus.tolist()
BR = list(zip(branch_df.l, branch_df.k, branch_df.m))


In [ ]:

# -----------------------------
# Model
# -----------------------------
m = pyo.ConcreteModel("AMPL_OPF_140")
m.BUS = pyo.Set(initialize=BUS, ordered=True)
m.BRANCH = pyo.Set(dimen=3, initialize=BR, ordered=True)

# adjacency (directed)
def out_arcs_init(mm, k):
    return [(l, k, j) for (l, kk, j) in mm.BRANCH if kk == k]

def in_arcs_init(mm, k):
    return [(l, i, k) for (l, i, kk) in mm.BRANCH if kk == k]

m.OUT = pyo.Set(m.BUS, dimen=3, initialize=out_arcs_init)
m.IN  = pyo.Set(m.BUS, dimen=3, initialize=in_arcs_init)

# YBUS set = diagonal + (k,m) + (m,k) for each branch
def ybus_init(mm):
    pairs = {(i, i) for i in mm.BUS}
    for (l, k, n) in mm.BRANCH:
        pairs.add((k, n))
        pairs.add((n, k))
    return sorted(pairs)

m.YBUS = pyo.Set(dimen=2, initialize=ybus_init)

def yrow_init(mm, k):
    return [n for (kk, n) in mm.YBUS if kk == k]

m.YROW = pyo.Set(m.BUS, initialize=yrow_init)


# -----------------------------
# Params (AMPL scaling)
# -----------------------------
m.bus_type = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_type)), within=pyo.Integers)

# powers scaled /100 like AMPL "let ... := .../100"
m.bus_p_gen  = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_p_gen / 100.0)))
m.bus_q_gen  = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_q_gen / 100.0)))
m.bus_q_min  = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_q_min / 100.0)))
m.bus_q_max  = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_q_max / 100.0)))
m.bus_p_load = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_p_load / 100.0)))
m.bus_q_load = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_q_load / 100.0)))

# valores iniciais do caso (AMPL usa como starting point)
m.bus_voltage0 = pyo.Param(
    m.BUS,
    initialize=dict(zip(bus_df.bus, bus_df.bus_voltage0)),
    mutable=False
)

m.bus_angle0_deg = pyo.Param(
    m.BUS,
    initialize=dict(zip(bus_df.bus, bus_df.bus_angle0_deg)),
    mutable=False
)


m.bus_g_shunt = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_g_shunt)))
m.bus_b_shunt0   = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_b_shunt0)))
m.bus_b_shunt_min = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_b_shunt_min)))
m.bus_b_shunt_max = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_b_shunt_max)))
m.bus_b_dispatch = pyo.Param(m.BUS, initialize=dict(zip(bus_df.bus, bus_df.bus_b_dispatch)), within=pyo.Integers)

m.branch_type = pyo.Param(m.BRANCH, initialize={t: int(v) for t, v in zip(BR, branch_df.branch_type)}, within=pyo.Integers)

m.branch_r = pyo.Param(m.BRANCH, initialize={t: float(v) for t, v in zip(BR, branch_df.branch_r)})
m.branch_x = pyo.Param(m.BRANCH, initialize={t: float(v) for t, v in zip(BR, branch_df.branch_x)})
m.branch_c = pyo.Param(m.BRANCH, initialize={t: float(v) for t, v in zip(BR, branch_df.branch_c)})

m.branch_tap0     = pyo.Param(m.BRANCH, initialize={t: float(v) for t, v in zip(BR, branch_df.branch_tap0)})
m.branch_tap_min0 = pyo.Param(m.BRANCH, initialize={t: float(v) for t, v in zip(BR, branch_df.branch_tap_min0)})
m.branch_tap_max0 = pyo.Param(m.BRANCH, initialize={t: float(v) for t, v in zip(BR, branch_df.branch_tap_max0)})

# degrees -> rad; AMPL uses: let branch_def := -branch_def*pi/180
m.branch_def0 = pyo.Param(m.BRANCH, initialize={t: -float(v) * math.pi / 180.0 for t, v in zip(BR, branch_df.branch_def0_deg)})
m.branch_def_min = pyo.Param(m.BRANCH, initialize={t:  float(v) * math.pi / 180.0 for t, v in zip(BR, branch_df.branch_def_min_deg)})
m.branch_def_max = pyo.Param(m.BRANCH, initialize={t:  float(v) * math.pi / 180.0 for t, v in zip(BR, branch_df.branch_def_max_deg)})

# series admittance
def _g(r, x):
    den = r*r + x*x
    return r/den

def _b(r, x):
    den = r*r + x*x
    return -x/den

m.branch_g = pyo.Param(m.BRANCH, initialize={t: _g(pyo.value(m.branch_r[t]), pyo.value(m.branch_x[t])) for t in m.BRANCH})
m.branch_b = pyo.Param(m.BRANCH, initialize={t: _b(pyo.value(m.branch_r[t]), pyo.value(m.branch_x[t])) for t in m.BRANCH})

# AMPL voltage limits (04_08_fpo_140.mod)
bus_voltage_min = {0: 0.95, 1: 0.95, 2: 0.95, 3: 1.00}
bus_voltage_max = {0: 1.05, 1: 1.05, 2: 1.05, 3: 1.00}
m.bus_voltage_min = pyo.Param([0,1,2,3], initialize=bus_voltage_min)
m.bus_voltage_max = pyo.Param([0,1,2,3], initialize=bus_voltage_max)

# AMPL p_gen bounds
m.p_gen_upper = pyo.Param(initialize=100.0)
m.p_gen_lower = pyo.Param(initialize=0.0)

# -----------------------------
# Shunt scaling (AMPL: let bus_b_shunt := bus_b_shunt0/10000)
# -----------------------------
m.bus_b_shunt0_scaled = pyo.Expression(m.BUS, rule=lambda mm, i: mm.bus_b_shunt0[i] / 10000.0)

# -----------------------------
# Variables
# -----------------------------
def V_bounds(mm, i):
    t = int(pyo.value(mm.bus_type[i]))
    return (float(pyo.value(mm.bus_voltage_min[t])), float(pyo.value(mm.bus_voltage_max[t])))

# Variáveis com inicialização do caso (em vez de 1.0 / 0.0)

m.bus_voltage = pyo.Var(
    m.BUS,
    bounds=V_bounds,
    initialize=lambda mm,i: float(pyo.value(mm.bus_voltage0[i]))
)

m.bus_angle = pyo.Var(
    m.BUS,
    initialize=lambda mm,i: float(pyo.value(mm.bus_angle0_deg[i])) * math.pi/180.0
)


def b_shunt_bounds(mm, i):
    return (float(pyo.value(mm.bus_b_shunt_min[i])), float(pyo.value(mm.bus_b_shunt_max[i])))

def b_shunt_init(mm, i):
    bmin = float(pyo.value(mm.bus_b_shunt_min[i]))
    bmax = float(pyo.value(mm.bus_b_shunt_max[i]))
    if abs(bmax - bmin) <= 1e-12:
        return bmin
    return float(pyo.value(mm.bus_b_shunt0_scaled[i]))

m.bus_b_shunt = pyo.Var(m.BUS, bounds=b_shunt_bounds, initialize=b_shunt_init)

# taps: use per-branch bounds from data
m.branch_tap = pyo.Var(
    m.BRANCH,
    bounds=lambda mm, l, k, n: (float(pyo.value(mm.branch_tap_min0[l,k,n])), float(pyo.value(mm.branch_tap_max0[l,k,n]))),
    initialize=lambda mm, l, k, n: float(pyo.value(mm.branch_tap0[l,k,n])),
)

m.branch_def = pyo.Var(
    m.BRANCH,
    bounds=lambda mm, l, k, n: (float(pyo.value(mm.branch_def_min[l,k,n])), float(pyo.value(mm.branch_def_max[l,k,n]))),
    initialize=lambda mm, l, k, n: float(pyo.value(mm.branch_def0[l,k,n])),
)

# -----------------------------
# Fixings (match AMPL)
# -----------------------------
# slack: type 3 -> angle fixed 0; voltage fixed at 1.00 by bounds (min=max=1.00) already, but keep explicit
for i in m.BUS:
    if int(pyo.value(m.bus_type[i])) == 3:
        m.bus_angle[i].fix(0.0)
        m.bus_voltage[i].fix(1.0)

# shunts: AMPL "freeze any dispatchable shunts": fix where bus_b_dispatch == 1
for i in m.BUS:
    if int(pyo.value(m.bus_b_dispatch[i])) == 1:
        m.bus_b_shunt[i].fix(pyo.value(m.bus_b_shunt0_scaled[i]))

# branch taps:
for (l, k, n) in m.BRANCH:
    bt = int(pyo.value(m.branch_type[l, k, n]))
    if bt in (0, 3):
        m.branch_tap[l, k, n].fix(1.0)
    if bt in (5, 6):
        # AMPL uses 1.02040, but the data bounds may be a hair different (rounding).
        # To avoid Pyomo "trivially infeasible" when bounds are fixed, clamp tap0 to [min,max].
        t0 = float(pyo.value(m.branch_tap0[l, k, n]))
        tmin = float(pyo.value(m.branch_tap_min0[l, k, n]))
        tmax = float(pyo.value(m.branch_tap_max0[l, k, n]))
        tfix = min(max(t0, tmin), tmax)
        m.branch_tap[l, k, n].fix(tfix)

# phase shifters:
for (l, k, n) in m.BRANCH:
    bt = int(pyo.value(m.branch_type[l, k, n]))
    if bt != 4:
        m.branch_def[l, k, n].fix(pyo.value(m.branch_def0[l, k, n]))


# -----------------------------
# YBUS (G,B) expressions (AMPL-equivalentes)
# -----------------------------
# map (k,n) -> list of real directed arcs stored as triples (l,f,t)
branch_km = defaultdict(list)
for (l, f, t) in m.BRANCH:
    branch_km[(f, t)].append((l, f, t))
    branch_km[(t, f)].append((l, f, t))

def G_rule(mm, k, n):
    if k == n:
        return (
            mm.bus_g_shunt[k]
            + sum(mm.branch_g[l,k,j] * mm.branch_tap[l,k,j]**2 for (l,k,j) in mm.OUT[k])
            + sum(mm.branch_g[l,j,k] for (l,j,k) in mm.IN[k])
        )
    if (k, n) not in branch_km:
        return 0.0

    expr = 0.0
    for (l, f, t) in branch_km[(k, n)]:
        if k == f and n == t:
            expr += -(mm.branch_g[l,f,t] * pyo.cos(mm.branch_def[l,f,t])
                      + mm.branch_b[l,f,t] * pyo.sin(mm.branch_def[l,f,t])) * mm.branch_tap[l,f,t]
        else:
            expr += -(mm.branch_g[l,f,t] * pyo.cos(mm.branch_def[l,f,t])
                      - mm.branch_b[l,f,t] * pyo.sin(mm.branch_def[l,f,t])) * mm.branch_tap[l,f,t]
    return expr

m.G = pyo.Expression(m.YBUS, rule=lambda mm, k, n: G_rule(mm, k, n))

def B_rule(mm, k, n):
    if k == n:
        return (
            mm.bus_b_shunt[k]
            + sum(mm.branch_b[l,k,j] * mm.branch_tap[l,k,j]**2 + mm.branch_c[l,k,j]/2.0 for (l,k,j) in mm.OUT[k])
            + sum(mm.branch_b[l,j,k] + mm.branch_c[l,j,k]/2.0 for (l,j,k) in mm.IN[k])
        )
    if (k, n) not in branch_km:
        return 0.0

    expr = 0.0
    for (l, f, t) in branch_km[(k, n)]:
        if k == f and n == t:
            expr += (mm.branch_g[l,f,t] * pyo.sin(mm.branch_def[l,f,t])
                     - mm.branch_b[l,f,t] * pyo.cos(mm.branch_def[l,f,t])) * mm.branch_tap[l,f,t]
        else:
            expr += (-mm.branch_g[l,f,t] * pyo.sin(mm.branch_def[l,f,t])
                     - mm.branch_b[l,f,t] * pyo.cos(mm.branch_def[l,f,t])) * mm.branch_tap[l,f,t]
    return expr

m.B = pyo.Expression(m.YBUS, rule=lambda mm, k, n: B_rule(mm, k, n))


In [ ]:
def Ploss_rule(mm, l, k, n):
    return (
        mm.branch_g[l,k,n] * (
            mm.bus_voltage[k]**2 * mm.branch_tap[l,k,n]**2
            + mm.bus_voltage[n]**2
            - 2 * mm.bus_voltage[k] * mm.bus_voltage[n]
              * mm.branch_tap[l,k,n]
              * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n] + mm.branch_def[l,k,n])
        )
    )

m.Plosses_branch = pyo.Expression(m.BRANCH, rule=Ploss_rule)

def Qloss_rule(mm, l, k, n):
    return (
        - mm.branch_b[l,k,n] * (
            mm.bus_voltage[k]**2 * mm.branch_tap[l,k,n]**2
            + mm.bus_voltage[n]**2
        )
        + 2 * mm.branch_b[l,k,n]
          * mm.bus_voltage[k] * mm.bus_voltage[n]
          * mm.branch_tap[l,k,n]
          * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n] + mm.branch_def[l,k,n])
    )

m.Qlosses_branch = pyo.Expression(m.BRANCH, rule=Qloss_rule)


# -----------------------------
# AC constraints (AMPL)
# -----------------------------
def p_load_rule(mm, k):
    if int(pyo.value(mm.bus_type[k])) != 0:
        return pyo.Constraint.Skip

    return (
        mm.bus_p_gen[k] - mm.bus_p_load[k]
        - sum(
            mm.bus_voltage[k] * mm.bus_voltage[n]
            * (
                mm.G[k,n] * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n])
              + mm.B[k,n] * pyo.sin(mm.bus_angle[k] - mm.bus_angle[n])
            )
            for n in mm.YROW[k]
        )
        == 0
    )

m.p_load = pyo.Constraint(m.BUS, rule=p_load_rule)

def q_load_rule(mm, k):
    if int(pyo.value(mm.bus_type[k])) != 0:
        return pyo.Constraint.Skip

    return (
        mm.bus_q_gen[k] - mm.bus_q_load[k]
        - sum(
            mm.bus_voltage[k] * mm.bus_voltage[n]
            * (
                mm.G[k,n] * pyo.sin(mm.bus_angle[k] - mm.bus_angle[n])
              - mm.B[k,n] * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n])
            )
            for n in mm.YROW[k]
        )
        == 0
    )

m.q_load = pyo.Constraint(m.BUS, rule=q_load_rule)

def q_inj_rule(mm, k):
    if int(pyo.value(mm.bus_type[k])) not in (2, 3):
        return pyo.Constraint.Skip

    Q = (
        mm.bus_q_load[k]
        + sum(
            mm.bus_voltage[k] * mm.bus_voltage[n]
            * (
                mm.G[k,n] * pyo.sin(mm.bus_angle[k] - mm.bus_angle[n])
              - mm.B[k,n] * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n])
            )
            for n in mm.YROW[k]
        )
    )
    return pyo.inequality(mm.bus_q_min[k], Q, mm.bus_q_max[k])

m.q_inj = pyo.Constraint(m.BUS, rule=q_inj_rule)

def p_inj_rule(mm, k):
    if int(pyo.value(mm.bus_type[k])) not in (2, 3):
        return pyo.Constraint.Skip

    P = (
        mm.bus_p_load[k]
        + sum(
            mm.bus_voltage[k] * mm.bus_voltage[n]
            * (
                mm.G[k,n] * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n])
              + mm.B[k,n] * pyo.sin(mm.bus_angle[k] - mm.bus_angle[n])
            )
            for n in mm.YROW[k]
        )
    )
    return pyo.inequality(mm.p_gen_lower * mm.bus_p_gen[k], P, mm.p_gen_upper * mm.bus_p_gen[k])

m.p_inj = pyo.Constraint(m.BUS, rule=p_inj_rule)


In [ ]:
def p_g_rule(mm, k):
    return (
        mm.bus_p_load[k]
        + sum(
            mm.bus_voltage[k] * mm.bus_voltage[n]
            * (
                mm.G[k,n] * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n])
              + mm.B[k,n] * pyo.sin(mm.bus_angle[k] - mm.bus_angle[n])
            )
            for n in mm.YROW[k]
        )
    )

def q_g_rule(mm, k):
    return (
        mm.bus_q_load[k]
        + sum(
            mm.bus_voltage[k] * mm.bus_voltage[n]
            * (
                mm.G[k,n] * pyo.sin(mm.bus_angle[k] - mm.bus_angle[n])
              - mm.B[k,n] * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n])
            )
            for n in mm.YROW[k]
        )
    )

m.p_g = pyo.Expression(m.BUS, rule=p_g_rule)
m.q_g = pyo.Expression(m.BUS, rule=q_g_rule)

def p_d_rule(mm, l, k, n):
    return (
        mm.branch_g[l,k,n] * mm.bus_voltage[k]**2 * mm.branch_tap[l,k,n]**2
      - mm.branch_g[l,k,n] * mm.bus_voltage[k] * mm.bus_voltage[n] * mm.branch_tap[l,k,n]
        * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n] + mm.branch_def[l,k,n])
      - mm.branch_b[l,k,n] * mm.bus_voltage[k] * mm.bus_voltage[n] * mm.branch_tap[l,k,n]
        * pyo.sin(mm.bus_angle[k] - mm.bus_angle[n] + mm.branch_def[l,k,n])
    )

def q_d_rule(mm, l, k, n):
    return (
      - mm.branch_b[l,k,n] * mm.bus_voltage[k]**2 * mm.branch_tap[l,k,n]**2
      - mm.branch_g[l,k,n] * mm.bus_voltage[k] * mm.bus_voltage[n] * mm.branch_tap[l,k,n]
        * pyo.sin(mm.bus_angle[k] - mm.bus_angle[n] + mm.branch_def[l,k,n])
      + mm.branch_b[l,k,n] * mm.bus_voltage[k] * mm.bus_voltage[n] * mm.branch_tap[l,k,n]
        * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n] + mm.branch_def[l,k,n])
    )

def p_r_rule(mm, l, k, n):
    return (
        mm.branch_g[l,k,n] * mm.bus_voltage[n]**2
      - mm.branch_g[l,k,n] * mm.bus_voltage[k] * mm.bus_voltage[n] * mm.branch_tap[l,k,n]
        * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n] + mm.branch_def[l,k,n])
      + mm.branch_b[l,k,n] * mm.bus_voltage[k] * mm.bus_voltage[n] * mm.branch_tap[l,k,n]
        * pyo.sin(mm.bus_angle[k] - mm.bus_angle[n] + mm.branch_def[l,k,n])
    )

def q_r_rule(mm, l, k, n):
    return (
      - mm.branch_b[l,k,n] * mm.bus_voltage[n]**2
      + mm.branch_g[l,k,n] * mm.bus_voltage[k] * mm.bus_voltage[n] * mm.branch_tap[l,k,n]
        * pyo.sin(mm.bus_angle[k] - mm.bus_angle[n] + mm.branch_def[l,k,n])
      + mm.branch_b[l,k,n] * mm.bus_voltage[k] * mm.bus_voltage[n] * mm.branch_tap[l,k,n]
        * pyo.cos(mm.bus_angle[k] - mm.bus_angle[n] + mm.branch_def[l,k,n])
    )

m.p_d = pyo.Expression(m.BRANCH, rule=p_d_rule)
m.q_d = pyo.Expression(m.BRANCH, rule=q_d_rule)
m.p_r = pyo.Expression(m.BRANCH, rule=p_r_rule)
m.q_r = pyo.Expression(m.BRANCH, rule=q_r_rule)

In [ ]:
EPS_ABS = 1e-16
def smooth_abs(x):
    return pyo.sqrt(x*x + EPS_ABS)

m.reactive_generation = pyo.Expression(
    expr=sum(smooth_abs(m.q_g[k]) for k in m.BUS)
)

In [ ]:
EPS_OBJ = 1e-12
sum_abs_P = sum(smooth_abs(m.Plosses_branch[l,k,n]) for (l,k,n) in m.BRANCH)
sum_abs_Q = sum(smooth_abs(m.Qlosses_branch[l,k,n]) for (l,k,n) in m.BRANCH)

m.obj = pyo.Objective(
    expr=0.05 * pyo.sqrt(sum_abs_P**2 + sum_abs_Q**2 + EPS_OBJ)
        + sum((m.bus_voltage[i] - 1.0)**2 for i in m.BUS),
    sense=pyo.minimize
)

In [ ]:
# -----------------------------
# Solve
# -----------------------------
solver = pyo.SolverFactory("ipopt")
# algumas opções úteis (opcional)
solver.options["tol"] = 1e-8
solver.options["max_iter"] = 5000

results = solver.solve(m, tee=True)




print("\nSolver status      :", results.solver.status)
print("Termination status :", results.solver.termination_condition)



Ipopt 3.12.13: tol=1e-08
max_iter=5000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1660
Number of nonzeros in inequality constraint Jacobian.:       58
Number of nonzeros in Lagrangian Hessian.............:    38784

Total number of variables............................:      281
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      142
                     variables with only up

In [ ]:
Vvals = [float(pyo.value(m.bus_voltage[i])) for i in m.BUS]
print("\nVmin/Vmax:", min(Vvals), max(Vvals))

base = 100.0  # MW/MVAr base as in AMPL prints (*100)
P_loss_total = sum(float(pyo.value(m.p_d[l,k,n] + m.p_r[l,k,n])) for (l,k,n) in m.BRANCH) * base
Q_loss_total = sum(abs(float(pyo.value(m.q_d[l,k,n] + m.q_r[l,k,n]))) for (l,k,n) in m.BRANCH) * base

P_gen_total = sum(float(pyo.value(m.p_g[k])) for k in m.BUS) * base
Q_gen_abs_total = sum(abs(float(pyo.value(m.q_g[k]))) for k in m.BUS) * base

Q_ind = sum(min(float(pyo.value(m.q_g[k])), 0.0) for k in m.BUS) * base
Q_cap = sum(max(float(pyo.value(m.q_g[k])), 0.0) for k in m.BUS) * base

print("\nObjective (pu):", float(pyo.value(m.obj)))
print(f"Total Active Power Losses   : {P_loss_total:.6f} MW")
print(f"Total Reactive Power Losses : {Q_loss_total:.6f} MVAr")
print(f"Total Active Generation     : {P_gen_total:.2f} MW")
print(f"Total Reactive Generation   : {Q_gen_abs_total:.2f} MVAr")
print(f"Total Inductive Generation  : {Q_ind:.2f} MVAr")
print(f"Total Capacitive Generation : {Q_cap:.2f} MVAr")
